In [1]:
# Code skeleton from Måns Andersson
# https://colab.research.google.com/github/MaansAndersson/colab_test/blob/main/FEniCSx.ipynb

import gmsh
import numpy as np
import petsc4py
import petsc4py.PETSc
import dolfinx

**B1.1 State and give a brief description of the basic steps for solving the Poisson equation using the FEM**




*   We define the equation (Poisson)
*   We define the boundary conditions
*   We decide on basis and test functions to use
*   The equation and boundary conditions are reformulated into a weak form
*   The weak form is discretised onto the mesh, represented by a combination of basis functions
*   We construct a linear system of equations from the above, applying the boundary conditions
*   Then, we can solve the linear system

**B1.2 What are the element and the assembly matrices? What is the relation between the element and assembly matrices?**

The element matrix is a matrix representing a single element, being constructed from a set of nodes. The assembly matrix is constructed from the element matrixes.

**B1.3 Use either a modified version of the code from the slides or some of the other tools shown during the lecture to create a suiting mesh.**

**B1.3.2 What are two data structures used for storing information about the mesh? What are the numbers of elements and nodes in the mesh? How is it found? HINT: study mesh.geometry.**
The two data structures used to represent the matrix are the el2no and no2xy matrices. The former represents each element as a sequence of nodes. The latter stores information about the x and y coordinates for each node.





In [2]:
from mpi4py import MPI


gmsh.initialize()
model = gmsh.model()
print(MPI.COMM_WORLD)
mesh_comm = MPI.COMM_WORLD
model_rank = 0
if mesh_comm.rank == model_rank:
  square = model.occ.addRectangle(0, 0, 0, 1, 1)
  circle = model.occ.addDisk(0, 0, 0, 0.5, 0.5)
  model_dim_tags = model.occ.cut([(2, square)], [(2, circle)])

  target_geometry = model_dim_tags[0][0]
  model.occ.synchronize()
  model.add_physical_group(2, [square])
  model.mesh.generate(2)
msh, mt, ft = dolfinx.io.gmshio.model_to_mesh(model, mesh_comm, model_rank)



Info    : Meshing 1D...                                                                                      
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 70%] Meshing curve 4 (Line)
Info    : [ 90%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000427542s, CPU 0.00017s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00321754s, CPU 0.001178s)
Info    : 81 nodes 165 elements


In [3]:
import dolfinx
with dolfinx.io.XDMFFile(msh.comm, "output/poisson.xdmf", "w") as file:
    file.write_mesh(msh)

In [28]:
# Define function space
import basix
import ufl

P1 = basix.ufl.element("Lagrange", msh.basix_cell() ,1)
W = dolfinx.fem.functionspace(msh, P1)

def f(x):
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) | np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0)

f_h = dolfinx.fem.Function(W)
f_h.interpolate(f)

# Define boundary conditions
facets = dolfinx.mesh.locate_entities_boundary(msh , msh.topology.dim - 1, lambda x: np.full(x.shape[1], True))
dofs = dolfinx.fem.locate_dofs_topological(W, 1, facets)
bc1 = dolfinx.fem.dirichletbc(f_h, dofs, W)
bc2 = dolfinx.fem.dirichletbc(f_h, dofs, W)
bcs = [ bc1 , bc2 ]
# Define variational problem
u = ufl.TrialFunction(W)
v = ufl.TestFunction(W)
dx = ufl.Measure("dx", domain=msh)
a = ufl.inner(ufl.grad(u), ufl.grad(v))*dx




L = ufl.inner(f, v)*dx
# Compute solution
petsc_opt = { " ... " }
solver = dolfinx.fem.petsc.LinearProblem (a , L, bcs, petsc_options = petsc_opt)
uh = solver.solve()
#Andersson (KTH) Lecture on FEM in FEniCSx 2024 18 / 29




TypeError: __init__(): incompatible function arguments. The following argument types are supported:
    1. __init__(self, g: ndarray[dtype=float64, writable=False, order='C'], dofs: ndarray[dtype=int32, writable=False, shape=(*), order='C'], V: dolfinx.cpp.fem.FunctionSpace_float64) -> None
    2. __init__(self, g: dolfinx.cpp.fem.Constant_float64, dofs: ndarray[dtype=int32, writable=False, shape=(*), order='C'], V: dolfinx.cpp.fem.FunctionSpace_float64) -> None
    3. __init__(self, g: dolfinx.cpp.fem.Function_float64, dofs: ndarray[dtype=int32, writable=False, shape=(*), order='C']) -> None
    4. __init__(self, g: dolfinx.cpp.fem.Function_float64, dofs: collections.abc.Sequence[ndarray[dtype=int32, writable=False, shape=(*), order='C']], V: dolfinx.cpp.fem.FunctionSpace_float64) -> None

Invoked with types: dolfinx.cpp.fem.DirichletBC_float64, dolfinx.cpp.fem.Function_float64, ndarray, dolfinx.cpp.fem.FunctionSpace_float64

In [ ]:
def outer_box(x):
    return np.isclose(x[0], 0.0)+np.isclose(x[0], 1.0)+np.isclose(x[1], 0.0)+np.isclose(x[1], 1.0)

dim_surface = (msh.topology.dim - 1)
facets0 = dolfinx.mesh.locate_entities_boundary(msh, dim=dim_surface, marker=outer_box)
dofs = dolfinx.fem.locate_dofs_topological(V=W, entity_dim=dim_surface, entities=facets0)
bc0 = dolfinx.fem.dirichletbc(0.0, dofs, W)

bb_tree = dolfinx.geometry.bb_tree(msh, 2)
x0 = np.array([[0.9, 0.9, 0.]], dtype=np.float64)
cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, x0)
cell = dolfinx.geometry.compute_colliding_cells(msh, cell_candidates, x0).array


NameError: name 'ufl' is not defined